# Question 1: Given some sample data, write a program to answer the following: click here to access the required data set

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


In [75]:
import pandas as pd
import numpy as np
#!pip install plotly==5.3.1
import plotly.graph_objects as go


In [76]:
shopify_df=pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")

In [77]:
shopify_df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


# Average order amount is 3145.12 , which is too high as comapared to the median value 284. Highly skewed distribution.

In [78]:
shopify_df.info()
#all are non null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


In [28]:
shopify_df[shopify_df.duplicated()]
#No duplicates

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at


In [52]:
from scipy.stats import ttest_1samp
import numpy as np

shopify_mean = np.mean(shopify_df)
print(shopify_mean)
tset, pval = ttest_1samp(shopify_mean, 3145)
print("p-values",pval)
if pval < 0.05:    # alpha value is 0.05 or 5%
   print("we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

order_id        2501.060405
shop_id           50.106362
user_id          849.918322
order_amount     754.091913
total_items        1.993980
dtype: float64
p-values 0.006912295063663728
we are rejecting null hypothesis


# Using One Sample T test - Checking whether statistically sample mean is different from population mean. As p value is less than 0.5 , therefore , there is significant difference between actual mean and expected mean.

In [79]:
import plotly.express as px
fig=px.scatter(shopify_df, x="user_id", y="order_amount")
fig.show()

# Clearly user_id 607 has got the maximum order amount of 704k , which is suspicious

In [80]:
import plotly.express as px
fig=px.scatter(shopify_df, x="user_id", y="order_amount",color="total_items",hover_data=["shop_id"])
fig.show()

# user 607 has bought 2000 items from shop number 42. Let's check further shop wise data

In [31]:
shopify_df["shop_id"].nunique()

100

In [32]:
avg_shop_order=shopify_df.groupby("shop_id")["order_amount"].mean()

In [33]:
fig = px.bar(avg_shop_order)
fig.show()

#Shop 42 and shop 78 accumulates the order amount of 284k which is 50% of the Total orderamount for all the shops. 

In [34]:
shopify_df=shopify_df.drop(shopify_df.loc[(shopify_df["shop_id"]==42)& (shopify_df["order_amount"]==704000)].sort_values(by=['created_at']).index)

# Removed the outliers where order amount > 704k

In [35]:
shopify_df[shopify_df["order_amount"]==704000]
#empty dataframe

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at


In [36]:
shopify_df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4983.000000,4983.000000,4983.000000,4983.000000,4983.00000
mean,2501.060405,50.106362,849.918322,754.091913,1.99398
std,1443.090253,29.051718,86.800308,5314.092293,0.98318
min,1.000000,1.000000,700.000000,90.000000,1.00000
25%,1250.500000,24.000000,776.000000,163.000000,1.00000
50%,2502.000000,50.000000,850.000000,284.000000,2.00000
75%,3750.500000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,154350.000000,8.00000


In [37]:
#Let's observer shop 78
shop_78_data=shopify_df[shopify_df["shop_id"]==78]

In [38]:
fig = px.pie(shop_78_data, values='order_amount', names='user_id', color='total_items')
fig.show()

In [39]:
shopify_df[shopify_df["user_id"]==834]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
643,644,68,834,272,2,debit,2017-03-11 17:00:15
1218,1219,41,834,236,2,credit_card,2017-03-14 23:07:07
1619,1620,95,834,336,2,debit,2017-03-05 15:16:05
1632,1633,73,834,330,2,cash,2017-03-27 14:03:19
1720,1721,95,834,168,1,debit,2017-03-30 8:49:20
2186,2187,55,834,342,2,credit_card,2017-03-18 23:47:06
2410,2411,6,834,374,2,debit,2017-03-15 9:56:43
2492,2493,78,834,102900,4,debit,2017-03-04 4:37:34
2573,2574,20,834,127,1,credit_card,2017-03-18 3:13:16
2591,2592,79,834,543,3,credit_card,2017-03-15 5:32:35


In [40]:
shopify_df[shopify_df["user_id"]==855]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
142,143,80,855,145,1,debit,2017-03-11 1:13:13
501,502,36,855,130,1,credit_card,2017-03-18 0:18:12
1665,1666,15,855,306,2,cash,2017-03-22 5:27:28
2270,2271,78,855,25725,1,credit_card,2017-03-14 23:58:22
2302,2303,84,855,306,2,debit,2017-03-12 9:22:39
2401,2402,61,855,158,1,cash,2017-03-05 18:26:31
2523,2524,51,855,187,1,cash,2017-03-08 20:39:45
2960,2961,43,855,543,3,credit_card,2017-03-29 21:42:54
3087,3088,45,855,284,2,credit_card,2017-03-25 21:35:25
3101,3102,78,855,51450,2,credit_card,2017-03-21 5:10:34


In [41]:
shop_78_data["order_amount"].mean()
#Average Order amount for Shop 78

49213.04347826087

In [42]:
other_shops_data=pd.concat([shopify_df,shop_78_data]).drop_duplicates(keep=False)
#Get data for shops other than 78
other_shops_data.shape

(4937, 7)

In [43]:
shopify_df.shape

(4983, 7)

In [44]:
other_shops_data["order_amount"].mean()
#Average order amount for other shops ( exclusing 78 ) 

302.58051448247926

# So ,another outlier is Shop number 78 

In [45]:
fig = px.bar(shop_78_data, x="order_id",y="payment_method")
fig.show()

In [46]:
other_shops_data.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734
std,1444.069407,29.061131,86.840313,160.804912,0.982821
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000


In [47]:
fig = px.box(other_shops_data, y="order_amount")
fig.show()

In [53]:
#Doing T - test with new mean value ( after removing outliers)
tset, pval = ttest_1samp(shopify_mean, 302.5)
print("p-values",pval)
if pval < 0.05:    # alpha value is 0.05 or 5%
   print("we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

order_id        2501.060405
shop_id           50.106362
user_id          849.918322
order_amount     754.091913
total_items        1.993980
dtype: float64
p-values 0.30720743260943023
we are accepting null hypothesis


In [61]:
fig = px.histogram(other_shops_data, x="order_amount")
fig.show()

In [59]:
fig = px.bar(other_shops_data, x="shop_id",y="order_amount")
fig.show()

In [71]:
import statistics
print(statistics.pstdev(shopify_df["order_amount"]))
print(statistics.pstdev(other_shops_data["order_amount"]))
# variance before removing outliers and after removing outliers

5313.559044163769
160.78862528979838


#Conclusion for Question 1: 
Average order mean value is 3145$ , however median values is 284$. The distribution is highly skewed. So, mean and standard deviation gives no information on assymtary. We will check first and third quaratile values.
Assumption is already given : As shop sells only affordable sneakers, so removed the two outliers , got the mean value as 302.5 . Validated with T- test. So, mean and variance are the two parameters to evaluate the spread of the data.

# Question 2: For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.




How many orders were shipped by Speedy Express in total?
What is the last name of the employee with the most orders?
What product was ordered the most by customers in Germany?

SELECT Count(OrderID) FROM
Orders O Inner Join Shippers S
ON O.ShipperID = S.ShipperID
Where ShipperName = 'Speedy Express'

Answer : 54

What is the last name of the employee with the most orders?

SELECT LastName FROM Employees 
WHERE EmployeeID = (Select EmployeeId from 
(SELECT EmployeeID, COUNT(OrderID) FROM 
Orders GROUP BY EmployeeID ORDER BY COUNT(OrderID) DESC LIMIT 1) Orders)


What product was ordered the most by customers in Germany?

Could not calculate as Customer data is missing